In [1]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('./data/train.csv')
df3 = pd.read_csv('./data/test.csv')


In [3]:
# split cabin into num let 
cabin_loc_let = []
cabin_loc_num = []
for i in df['Cabin']:
    x = str(i)
    cabin_loc_let.append(x[0])
    if x[1] == 'a':
        cabin_loc_num.append(0)
    else:
        cabin_loc_num.append(x[1:])
df['cabin_loc_let'] = cabin_loc_let
df['cabin_loc_num'] = cabin_loc_num

cll_le = preprocessing.LabelEncoder() 
cll_enc = cll_le.fit_transform(df['cabin_loc_let'])
enc = OneHotEncoder(sparse = True)
cll_enc = cll_enc.reshape(-1,1)
cll_onehot = enc.fit_transform(cll_enc).toarray()
cll_cat = enc.categories_
cll_cat = np.asarray(cll_cat)
cll_cat = cll_cat.reshape(-1,1)
cll_inv = cll_le.inverse_transform(cll_cat)
cll_df = pd.DataFrame(data = cll_onehot, columns = cll_inv)
df2 = pd.concat([df, cll_df], axis = 1) 

mean_age = df2['Age'].mean()
mean_fare = df2['Fare'].mean()
df2['Age'] = df2['Age'].fillna(value = mean_age)
df2['Fare'] = df2['Fare'].fillna(value = mean_fare)
df2['Embarked'] = df2['Fare'].fillna(value = 'n')
df2.columns[df2.isna().any()].tolist()

df2['A'] = df2['A'].astype(str)
df2['B'] = df2['B'].astype(str)
df2['C'] = df2['C'].astype(str)
df2['D'] = df2['D'].astype(str)
df2['E'] = df2['E'].astype(str)
df2['F'] = df2['F'].astype(str)
df2['G'] = df2['G'].astype(str)
df2['T'] = df2['T'].astype(str)
df2['n'] = df2['n'].astype(str)
df2['Embarked'] = df2['n'].astype(str)
df2.pop('Ticket')
df2.pop('Cabin')
df2.pop('cabin_loc_num')

c:\users\jimmy\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0        12239
1            0
2            0
3            0
4            0
         ...  
99995    17243
99996        0
99997        0
99998        0
99999        0
Name: cabin_loc_num, Length: 100000, dtype: object

In [4]:
y = df2.pop('Survived')
X = df2

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

CATEGORICAL_COLUMNS = ['Pclass', 'Sex', 'SibSp', 'Parch', 'A',
                       'B', 'C', 'D', 'E', 'F', 'G', 'T', 'n', 'Embarked']


NUMERIC_COLUMNS = ['Age', 'Fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = df2[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

In [7]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(X_train, y_train)
test = make_input_fn(X_test, y_test, num_epochs=1, shuffle=False)

In [8]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
tf_model = linear_est.train(train_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Jimmy\\AppData\\Local\\Temp\\tmphb1amnrd', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initializ

In [9]:
result = tf_model.predict(test)
# result = linear_est.evaluate(eval_input_fn)


In [15]:
result2 = linear_est.evaluate(test)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-04-07T17:46:12Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Jimmy\AppData\Local\Temp\tmphb1amnrd\model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 4.52525s
INFO:tensorflow:Finished evaluation at 2021-04-07-17:46:16
INFO:tensorflow:Saving dict for global step 25000

In [ ]:
# # save model as file
# import pickle

# filename = 'tf_model.sav'
# pickle.dump(tf_model, open(filename, 'wb'))

# print(done)

# filename = 'clf150.sav'
# pickle.dump(clf100, open(filename, 'wb'))

# filename = 'clf200.sav'
# pickle.dump(clf500, open(filename, 'wb'))

# filename = 'clf250.sav'
# pickle.dump(clf1000, open(filename, 'wb'))

In [ ]:
# use model for test